In [31]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle
from langchain import hub
prompt = hub.pull("mehdixlabetix/rag-law-fr")



# Document loader

In [2]:
import glob

pdf_files = glob.glob("src/*.pdf")
pages = []

for pdf_file in pdf_files:
    pages.extend(PyPDFLoader(pdf_file).load_and_split())

for page in pages:
    page.page_content = page.page_content.replace("Imprimerie Officielle de la République Tunisienne", "")


# Splitter


In [5]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
len(chunks)


28127

# Vectorstores
## for now BAAI/bge-m3 is the best embedder

#### In the cloud (pinecone)

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "9721efa4-ff98-4bc5-9b28-93919d1657a5"
# embedder = HuggingFaceEmbeddings(
#     model_name = 'dangvantuan/sentence-camembert-large'
# )

index_name = "pfa"

#docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)
#docsearch = PineconeVectorStore(index_name=index_name,embedding=embedder)

#### Locally (chroma)

In [2]:

embedder = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-m3"
)
persist_directory = 'docs/chroma_bge/'
#vectordb.delete_collection()
# vectordb = Chroma.from_documents(
#     documents=chunks,
#     embedding=embedder,
#     persist_directory=persist_directory
# ) 
# vectordb.persist()
vectordb = Chroma(persist_directory="docs/chroma_bge/",embedding_function=embedder)
print(vectordb._collection.count())

28127


# Retrieval
### 3 methodes

In [29]:

query="quelle est la punition du meurtre?"

#méthode 1 avec retriever
# retriever = vectordb.as_retriever(search_type="mmr")
# matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)

#méthode 2 avec mmr
found_docs = vectordb.max_marginal_relevance_search(query, k=4, fetch_k=100)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")

#méthode 3 avec similarity search
# sim_docs = vectordb.similarity_search_with_score(query, k=5)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)
    

1. puni de mort. Si c'est un civil, la peine est réduite de moitié et la peine de mort est ramenée à celle de quinze ans d'emprisonnement. 


2. Article 201.- Est puni de mort, quiconque aura, par quelque 
moyen que ce soit, commis volontairement et avec préméditation un 
homicide.  
Article 202.- La préméditation consiste dans le dessein, formé 
avant l'action, d'attentat à la personne d'autrui. 
Article 203.- Est puni de mort, l’auteur de parricide.  
Est qualifié parricide, le meurtre des ascendants quel qu’en soit le 
degré. 


3. Article 211 (Modifié par la loi n° 89-23 du 27 février 1989).- Est 
puni de deux ans de prison, le meurtre commis par la mère sur son enfant à sa naissance ou immédiatement après. 


4. peines plus sévères prévues par d'autres textes. 




# Prompt Engineering and generation

In [32]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

final_prompt = prompt.invoke({"context":format_docs(found_docs), "question":query})

print(final_prompt)

messages=[HumanMessage(content=" Tu es un assistant juridique spécialisé dans la loi en Tunisie. Ta mission est de répondre aux questions des gens sur différents aspects juridiques ,en te limitant aux informations générales et en évitant les cas sensibles ou extrêmes. Si une question dépasse ton champ d'expertise ou si elle concerne un sujet délicat, tu dois informer l'utilisateur que tu ne peux pas fournir d'aide spécifique dans ce cas. Utilise les pièces suivantes du contexte pour répondre. Utilise un langage simple et accessible pour garantir que tout le monde puisse comprendre tes réponses. Gardez votre réponse brève et efficace maximum quatre ou cinq phrases.\xa0Tu dois répondre en français.\n Question: quelle est la punition du meurtre?.\nContexte: puni de mort. Si c'est un civil, la peine est réduite de moitié et la peine de mort est ramenée à celle de quinze ans d'emprisonnement.\n\nArticle 201.- Est puni de mort, quiconque aura, par quelque \nmoyen que ce soit, commis volontai

In [33]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")

llm.invoke(final_prompt)

" Selon la loi en Tunisie, le meurtre est puni de mort (Article 201). Cependant, si c'est un civile, la peine peut être réduite à quinze ans d'emprisonnement (Article 201, context provided)."